# ERA5 Correction via MintPy

In [1]:
# can try mintpy approach
#!smallbaselineApp.py FernandinaSenDT128.txt --dostep correct_troposphere
# can try barebones pyaps?
#import PyAPS
#!tropo_pyaps3.py -d 20151002 20151003 --hour 12 -m ERA5 -g inputs/geometryRadar.h5

In [2]:
#https://github.com/ASFHyP3/hyp3-docs/blob/develop/docs/tutorials/hyp3_insar_stack_for_ts_analysis.ipynb

In [59]:
import mintpy
from pathlib import Path
from dateutil.parser import parse as parse_date
from pathlib import Path
from typing import List, Union
from osgeo import gdal
from pathlib import Path
from typing import List, Union
import os
import glob

In [4]:
project_name = 'training'
work_dir = Path.cwd() / project_name
data_dir = work_dir / 'data'

In [5]:
data_dir.mkdir(parents=True, exist_ok=True)

In [6]:
def get_common_overlap(file_list: List[Union[str, Path]]) -> List[float]:
    """Get the common overlap of  a list of GeoTIFF files
    
    Arg:
        file_list: a list of GeoTIFF files
    
    Returns:
         [ulx, uly, lrx, lry], the upper-left x, upper-left y, lower-right x, and lower-right y
         corner coordinates of the common overlap
    """
    
    corners = [gdal.Info(str(dem), format='json')['cornerCoordinates'] for dem in file_list]

    ulx = max(corner['upperLeft'][0] for corner in corners)
    uly = min(corner['upperLeft'][1] for corner in corners)
    lrx = min(corner['lowerRight'][0] for corner in corners)
    lry = max(corner['lowerRight'][1] for corner in corners)
    return [ulx, uly, lrx, lry]

def clip_hyp3_products_to_common_overlap(data_dir: Union[str, Path], overlap: List[float]) -> None:
    """Clip all GeoTIFF files to their common overlap
    
    Args:
        data_dir:
            directory containing the GeoTIFF files to clip
        overlap:
            a list of the upper-left x, upper-left y, lower-right-x, and lower-tight y
            corner coordinates of the common overlap
    Returns: None
    """

    
    files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif']

    for extension in files_for_mintpy:

        for file in data_dir.rglob(f'*{extension}'):

            dst_file = file.parent / f'{file.stem}_clipped{file.suffix}'

            gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=overlap)

In [7]:
files = data_dir.glob('*/*_dem.tif')
overlap = get_common_overlap(files)
clip_hyp3_products_to_common_overlap(data_dir, overlap)

In [8]:
mintpy_config = work_dir / 'mintpy_config.txt'
mintpy_config.write_text(
f"""
mintpy.load.processor        = hyp3
##---------interferogram datasets:
mintpy.load.unwFile          = {data_dir}/*/*_unw_phase_clipped.tif
mintpy.load.corFile          = {data_dir}/*/*_corr_clipped.tif
##---------geometry datasets:
mintpy.load.demFile          = {data_dir}/*/*_dem_clipped.tif
mintpy.load.incAngleFile     = {data_dir}/*/*_lv_theta_clipped.tif
mintpy.load.azAngleFile      = {data_dir}/*/*_lv_phi_clipped.tif
mintpy.load.waterMaskFile    = {data_dir}/*/*_water_mask_clipped.tif
""")

928

In [9]:
!smallbaselineApp.py --dir {work_dir} {mintpy_config} --dostep load_data

MintPy version v1.3.3, date 2022-04-14
--RUN-at-2022-05-26 15:21:13.801276--
Current directory: /mnt/working/egagli/sw/repos/AMATH563-InSAR-denoiser/data-processing
Run routine processing with smallbaselineApp.py on steps: ['load_data']
Remaining steps: ['modify_network', 'reference_point', 'quick_overview', 'correct_unwrap_error', 'invert_network', 'correct_LOD', 'correct_SET', 'correct_troposphere', 'deramp', 'correct_topography', 'residual_RMS', 'reference_date', 'velocity', 'geocode', 'google_earth', 'hdfeos5']
--------------------------------------------------
Project name: mintpy_config
Go to work directory: /mnt/working/egagli/sw/repos/AMATH563-InSAR-denoiser/data-processing/training
copy default template file /mnt/working/egagli/sw/miniconda3/envs/mintpy/lib/python3.10/site-packages/mintpy/defaults/smallbaselineApp.cfg to work directory
read custom template file: /mnt/working/egagli/sw/repos/AMATH563-InSAR-denoiser/data-processing/training/mintpy_config.txt
update default templ

In [79]:
directories = [os.path.abspath(x[0]) for x in os.walk(data_dir)]
directories.remove(os.path.abspath(data_dir)) # If you don't want your main directory included
directories.remove('/mnt/working/egagli/sw/repos/AMATH563-InSAR-denoiser/data-processing/training/data/S1AA_20190505T135154_20190517T135155_VVP012_INT80_G_weF_5C8F/.ipynb_checkpoints')
geom_path = '../../inputs/geometryGeo.h5'

In [82]:
for i in directories:
    os.chdir(i)         # Change working Directory
    unw_filename = glob.glob('*_unw_phase.tif')[0]
    #print(unw_filename)
    #!pwd
    !tropo_pyaps3.py -f $unw_filename -g $geom_path
    break

weather model: ERA5 - dry (hydrostatic) and wet delay
weather directory: /mnt/working/egagli/sw/repos/AMATH563-InSAR-denoiser/data-processing/training/data/S1AA_20190505T135154_20190517T135155_VVP012_INT80_G_weF_5C8F
output tropospheric delay file: ../../inputs/ERA5.h5
output corrected time-series file: S1AA_20190505T135154_20190517T135155_VVP012_INT80_G_weF_5C8F_unw_phase_ERA5.tif
read dates/time info from file: S1AA_20190505T135154_20190517T135155_VVP012_INT80_G_weF_5C8F_unw_phase.tif
Traceback (most recent call last):
  File "/mnt/working/egagli/sw/miniconda3/envs/mintpy/bin/tropo_pyaps3.py", line 8, in <module>
    sys.exit(main())
  File "/mnt/working/egagli/sw/miniconda3/envs/mintpy/lib/python3.10/site-packages/mintpy/tropo_pyaps3.py", line 834, in main
    read_inps2date_time(inps)
  File "/mnt/working/egagli/sw/miniconda3/envs/mintpy/lib/python3.10/site-packages/mintpy/tropo_pyaps3.py", line 217, in read_inps2date_time
    inps.date_list = ptime.yyyymmdd(atr['DATE12'].split('-'